# Andres Graterol - 4031393 - Fall 22
# Homework 5

## Problem 1

In [4]:
import numpy as np

# Initialize the weights 
number_of_classes_M = 2 
number_of_features_F = 5
# W is of size M x F (Each class (M) has 5 classes (F))

# Initialize the weights from a uniform pdf with bounds [-1, 1]
initial_weights = np.random.uniform(low=-1, high=1, 
                                   size=(number_of_classes_M, number_of_features_F))

In [6]:
# For some opbservation vector, x, of size F (number of features)
# output is given by y(x) = Wx = (y1(x), y2(x))^T
# Output is of size (2,) -> (2, 5) X (5,)

print(initial_weights)

# Number of data points 
N = 1000

# TODO: NEED TO ASK ABOUT THE INSTRUCTIONS FOR THIS PART
#       SPECIFICALLY THE MEAN, VARIANCE, AND CLASS
# FOR LABELS BELONGING TO CLASS 0??????????????????????
# ARE BOTH OF SIZE 1000? OR 1000 TOTAL?
training_data_points_class0 = np.random.normal(loc=0, scale=1, size=(N,))
training_data_points_class1 = np.random.normal(loc=0.1, scale=1, size=(N,))

# COMBINE THESE TWO ARRAYS 

print(training_data_points)

# TODO: CREATE THE TRUE LABELS
# Vector will be of size (N, 2)
# DO WE NEED ONE OR TWO OF THESE???

lr = 0.00005 
iterations = 50

[[-0.88474183  0.59104769  0.02612412 -0.08823362  0.96434173]
 [-0.90222221 -0.70516565  0.98819435 -0.74085544  0.13329158]]
[-7.84152782e-01  9.47504703e-01  1.32076062e+00 -3.89744407e-01
  2.87171198e-01  6.80943188e-01  3.73154842e-01 -1.25884772e-02
 -8.34353599e-01  1.82872680e-01 -4.56614574e-01 -1.58310559e-01
  1.76283461e-01 -2.16106425e+00  1.07561851e+00 -2.67625108e-01
  1.81669038e-01 -1.39348570e+00  6.97215918e-01  2.32023739e-01
  1.11146446e+00 -6.11092612e-02 -4.74850575e-01 -1.10421838e+00
 -3.82827210e+00 -4.95546724e-01 -9.40048060e-01  4.44139035e-01
 -4.82408795e-01  5.56720619e-01 -1.00034189e+00 -2.99902097e-01
 -3.44247360e-01  1.40775194e-01  1.29655825e+00  1.44197301e+00
 -1.97528138e+00 -7.97611250e-01  9.16899075e-01  9.38297803e-01
 -6.58163758e-02 -3.18615042e+00  1.87694073e+00 -9.59504430e-01
  1.75569901e-01 -1.35168602e-01 -9.83268941e-01  1.45166910e+00
 -2.23647316e-01 -5.90170249e-01 -5.47503538e-01  1.09652209e+00
 -3.22430812e-01  1.47471294

In [ ]:
# New weights = weights from previous step - lr(gradients)
for i in range(iterations):
    

# Plot the training error (total MSE) 
# and classification accuracy w.r.t the number of iterations 


In [ ]:
def compute_error()

## Problem 2